In [1]:
%pip install pandas
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Ejercicio 5: con variables broadcast mostrar el nombre de los usuarios segun su id

In [29]:
import pyspark
from pyspark.sql import SparkSession
from pymongo import MongoClient
import pandas as pd

# Crea una SparkSession
spark = SparkSession.builder \
    .appName("MongoSparkConnector2") \
    .getOrCreate()

# Conecta a MongoDB usando pymongo
client = MongoClient("mongodb://172.17.0.3:27017/")
db = client['docker']
collection = db['perfilesmongo']

# Extrae los datos desde MongoDB
mongo_data = list(collection.find())

# Convierte los datos a un DataFrame de pandas
pdf = pd.DataFrame(mongo_data)

# Elimina la columna '_id' si es necesario, ya que no es serializable por defecto en Spark
if '_id' in pdf.columns:
    pdf = pdf.drop(columns=['_id'])

# Convierte el DataFrame de pandas a un DataFrame de Spark
df = spark.createDataFrame(pdf)

# Crear un RDD con nombres de varias ciudades
ciudades_rdd = [
    'Barcelona', 'Valladolid'
]



# Crear una variable broadcast
broadcast_cities = spark.sparkContext.broadcast(ciudades_rdd)

# Seleccionar solo la columna 'location'
rdd = df.select("jobtitle", "location").rdd

# Filtrar ubicaciones que están en la lista broadcast
rdd_filtrado = rdd.map(lambda x: (x['jobtitle'], x['location'])) 
# Filtrar el RDD usando la lista broadcast
ciudades_importantes_rdd = rdd_filtrado.filter(lambda x: x[1] in broadcast_cities.value)





# Recoger y mostrar los resultados
results = ciudades_importantes_rdd.collect()

print(results)

# Detener la SparkSession
spark.stop()


[('Full Stack Engineer | NodeJS | Express | React | Angular | MongoDB | MySQL | PHP', 'Barcelona'), ('Front End Developer / Back End Developer | React | Redux | NodeJS | MongoDB | Typescript | Jest | Cypress | HTML | CSS | JS | SASS', 'Barcelona'), ('Data analyst - Data Scientist | Python • SQL • MongoDB • Tableau • Machine Learning', 'Barcelona'), ('Full-Stack Developer: Javascript (ES6) | ReactJS | NodeJS | Java | Springboot | MySQL | MongoDB', 'Barcelona'), ('Junior Backend Developer | Java & Spring Boot | MySQL | MongoDB | Git | Postman', 'Barcelona'), ('Helping organizations unlock the power of Data', 'Barcelona'), ('Full Stack Developer | HTML5 | CSS3 | SCSS | ReactJS | Redux | Next JS | Javascript | Typescript | Git | NodeJS | MySQL | MongoDB | Java | ELM | Rescript | Functional Programming', 'Barcelona'), ('Developer | JavaScript | TypeScript | Express | Next | Nest | Node | React | Svelte | MySQL | MongoDB | CSS | HTML | GraphQL | Git | TensorFlow | Rest | Docker | Kubernetes 